<a href="https://colab.research.google.com/github/NaveenR2005/reg-no-2023502006/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install required packages
!pip install -q transformers gradio

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gradio as gr

# Load the model and tokenizer
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Store chat history across calls
chat_history = []

# Define the chatbot function
def chat(user_input):
    global chat_history
    # Encode user input and append end-of-string token
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

    # Concatenate chat history with new input
    bot_input_ids = torch.cat(chat_history + [new_input_ids], dim=-1) if chat_history else new_input_ids

    # Generate a response
    output_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # Save chat history
    chat_history.append(new_input_ids)
    chat_history.append(output_ids[:, bot_input_ids.shape[-1]:])

    # Decode the response
    response = tokenizer.decode(output_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    return response

# Gradio interface
gr.Interface(fn=chat,
             inputs=gr.Textbox(lines=2, placeholder="Say something..."),
             outputs="text",
             title="DialoGPT Chatbot",
             description="A simple chatbot using Microsoft's DialoGPT-small model.").launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://912aee22ee20f7b8fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
